In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import os
import glob
from pathlib import Path

In [2]:
# get the pdf files path
docs_dir = "../documents"
pdf_files = glob.glob(f"{docs_dir}/*.pdf")


# add pdf files to docs after loading with pdfLoader.
docs = []
for f in pdf_files:
    docs += PyPDFLoader(f).load()
    

In [ ]:
# split text into chunks

HEADING_SEPARATORS = [
    r"\nSITUATION\s+\d+\.\d+[^\n]*\n",     # Situation Handbook entries
    r"\nRULE\s+\d+[^\n]*\n",               # Rulebook: "RULE 60 High-sticking"
    r"\nSECTION\s+\d+[^\n]*\n",            # "SECTION 08. STICK INFRACTIONS"
    r"\n[A-Z][A-Z &/’'–\-]{4,}\n",         # ALL-CAPS headings like "EQUIPMENT"
    r"\n{2,}",                             # blank lines (paragraph breaks)
    r"\n",                                 # single newline
    r" "                                   # as last resort
]

coarse_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1800,
    chunk_overlap=50,
    separators=HEADING_SEPARATORS,
    keep_separator=True,
    is_separator_regex=True
    )


fine_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
    separators=[r"\n{2,}", r"\n", r" "],
    keep_separator=False,
    is_separator_regex=True
)

coarse_chunks = coarse_text_splitter.split_documents(docs)
final_chunks = []
for d in coarse_chunks:
    
    # split content of each coarse chunk further
    sub_docs = fine_splitter.split_documents([d])
    final_chunks.extend(sub_docs)



print(f"Coarse chunks: {len(coarse_chunks)} | Final chunks: {len(final_chunks)}")


Coarse chunks: 843 | Final chunks: 1685


In [4]:
# get embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [5]:
# Vector Store (Chroma)
vector_store = Chroma.from_documents(final_chunks, embeddings, persist_directory="../vectorstore")

In [6]:
# retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [7]:
llm = OllamaLLM(model="llama3.1:8b")

prompt = ChatPromptTemplate.from_template(
    """You are a hockey rules assistant.
    Use the context to answer. Cite sources as (page/section) from metadata.
    Question: {question}
    Context: {context}
    Answer with bullet points and citations."""
)


def format_docs(docs):
    """Format a list of documents into a readable string with sources and page numbers."""
    formatted_docs = []

    for d in docs:
        content = d.page_content
        source = d.metadata.get("source", "?")
        page = d.metadata.get("page", "?")

        formatted_doc = f"- {content}\n  [source: {source}, page {page}]"
        formatted_docs.append(formatted_doc)

    return "\n\n".join(formatted_docs)


rag_chain = (
    {"context": (lambda x: x["question"]) | retriever | format_docs, "question": lambda x: x["question"]}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
q = "Does a player have to play the puck if possible during an icing situation?"
print(rag_chain.invoke({"question": q}))


Here are the answers to your question:

* **Yes**, a player has the option to play the puck if possible during an icing situation.
* The relevant rule is Rule 81.3 (page 135), which states: "If a player makes a play on the puck, or skates in the direction of the puck on an icing at any time, the potential icing shall NOT be called, and play shall continue."
* This means that if a defending team player attempts to play the puck or skate towards it during an icing situation, the potential icing shall not be called, even with an extra attacker (as per Situation 81.13, page 137).
* However, if a defending team player does not attempt to play the puck and instead bats the puck out of bounds with their hand from behind the blue line, this would be considered icing under Rule 81.1 (page 137).


• If the puck goes outside the playing area directly off a face-off, the face-off remains in the same spot (page 146).
• No penalty is assessed to either team for delaying the game if the puck goes outside the playing area directly off a face-off.
• If the puck is shot and makes contact with the gloves or body of a player hanging over the bench, or enters the players' bench through an open bench door, the face-off takes place at the nearest face-off spot in the zone from [../documents\2025-26_iihf_rulebook_30062025-v1.pdf, page 146].

Note: There is no specific rule stating where the face-off should take place after a puck out of bounds, other than the conditions mentioned above. The general principle is to give the offending team the least amount of "territorial advantage" (page 146).